In [6]:
try:
    from google.colab import drive
    drive.mount('/content/drive')
    G_path = '/content/drive/MyDrive/DataSources/Project_Data'
except:
    G_path = '/content/drive/MyDrive/DataSources/Project_Data'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
import torch.nn as nn
import torch.nn.functional as F
import torch
from torch.utils.data.sampler import WeightedRandomSampler
from torch.utils.data import SubsetRandomSampler, DataLoader, SequentialSampler
import torchvision
import numpy as np
from torchvision import transforms
from matplotlib import docstring
from PIL import Image


class ConvAutoEncoder(nn.Module):
    def __init__(self):
        super().__init__()

        self.activation = F.leaky_relu
        self.sig = torch.sigmoid
        self.pool = nn.MaxPool2d(kernel_size=2, return_indices=True)
        self.unpool = nn.MaxUnpool2d(kernel_size=2)
        self.normalization = nn.BatchNorm2d(3)

        #encoder
        self.conv1E = nn.Conv2d(3, 180, 4, 2)
        self.conv2E = nn.Conv2d(180, 120, 4, dilation=2)
        self.conv3E = nn.Conv2d(120, 90, 4)
        self.conv4E = nn.Conv2d(90, 60, 3, dilation=2)
        self.conv5E = nn.Conv2d(60, 30, 3)
        self.conv6E = nn.Conv2d(30, 3, 3)

        #decoder
        self.conv6D = nn.ConvTranspose2d(3, 30, 3)
        self.conv5D = nn.ConvTranspose2d(30, 60, 4)
        self.conv4D = nn.ConvTranspose2d(60, 90, 4)     
        self.conv3D = nn.ConvTranspose2d(90, 120, 4)
        self.conv2D = nn.ConvTranspose2d(120, 180, 6)
        self.conv1D = nn.ConvTranspose2d(180, 3, 6, 2)
        
    def forward(self, x):
      x = self.encode(x)
      enc = x
      x = self.decode(x)
      return x, enc

    def encode(self, x):
        x = self.normalization(x)
        x = self.activation(self.conv1E(x))
        x = self.activation(self.conv2E(x))
        x = self.activation(self.conv3E(x))
        x = self.activation(self.conv4E(x))
        x = self.activation(self.conv5E(x))
        x = self.sig(self.conv6E(x))
        return x
    
    def decode(self, x):
        x = self.activation(self.conv6D(x))
        x = self.activation(self.conv5D(x))
        x = self.activation(self.conv4D(x))
        x = self.activation(self.conv3D(x))
        x = self.activation(self.conv2D(x))
        x = self.sig(self.conv1D(x))
        return x

model = ConvAutoEncoder()

model = ConvAutoEncoder().double()
model = model.to("cuda")
model.load_state_dict(torch.load(G_path + '/08_Saved_Models_Outpus/AutoEncoder/CAE_NoPool'))#, map_location=torch.device('cpu')))
model.eval()

In [9]:
dataset = torchvision.datasets.ImageFolder(root= G_path + '/01_Augmented_Imaged/train', transform=transforms.ToTensor())

for i in dataset.imgs:
    print(i[0])
    a = Image.open(i[0])
    a = np.asarray([np.asarray(a)], dtype=np.float64)
    input = torch.from_numpy(a).permute(0,3,1,2).to("cuda")
    
    out, latent_space_images = model.forward(input/255)
    Image.fromarray((latent_space_images[0].cpu().detach().numpy().transpose(1,2,0)*255).astype(np.uint8)).save(i[0].replace("Augmentation", "dataset_splitted_augmented"))


/content/drive/MyDrive/Augmentation/train/0/AUG_ISIC_0024329.jpg
/content/drive/MyDrive/Augmentation/train/0/AUG_ISIC_0024418.jpg
/content/drive/MyDrive/Augmentation/train/0/AUG_ISIC_0024450.jpg
/content/drive/MyDrive/Augmentation/train/0/AUG_ISIC_0024470.jpg
/content/drive/MyDrive/Augmentation/train/0/AUG_ISIC_0024511.jpg
/content/drive/MyDrive/Augmentation/train/0/AUG_ISIC_0024517.jpg
/content/drive/MyDrive/Augmentation/train/0/AUG_ISIC_0024562.jpg
/content/drive/MyDrive/Augmentation/train/0/AUG_ISIC_0024710.jpg
/content/drive/MyDrive/Augmentation/train/0/AUG_ISIC_0024771.jpg
/content/drive/MyDrive/Augmentation/train/0/AUG_ISIC_0024800.jpg
/content/drive/MyDrive/Augmentation/train/0/AUG_ISIC_0024913.jpg
/content/drive/MyDrive/Augmentation/train/0/AUG_ISIC_0024925.jpg
/content/drive/MyDrive/Augmentation/train/0/AUG_ISIC_0025029.jpg
/content/drive/MyDrive/Augmentation/train/0/AUG_ISIC_0025089.jpg
/content/drive/MyDrive/Augmentation/train/0/AUG_ISIC_0025130.jpg
/content/drive/MyDrive/Au

In [10]:
dataset = torchvision.datasets.ImageFolder(root= G_path + '/01_Augmented_Imaged/valid', transform=transforms.ToTensor())

for i in dataset.imgs:
    print(i[0])
    a = Image.open(i[0])
    a = np.asarray([np.asarray(a)], dtype=np.float64)
    input = torch.from_numpy(a).permute(0,3,1,2).to("cuda")
    
    out, latent_space_images = model.forward(input/255)
    Image.fromarray((latent_space_images[0].cpu().detach().numpy().transpose(1,2,0)*255).astype(np.uint8)).save(i[0].replace("Augmentation", "dataset_splitted_augmented"))

/content/drive/MyDrive/Augmentation/valid/0/ISIC_0024522.jpg
/content/drive/MyDrive/Augmentation/valid/0/ISIC_0024654.jpg
/content/drive/MyDrive/Augmentation/valid/0/ISIC_0024707.jpg
/content/drive/MyDrive/Augmentation/valid/0/ISIC_0025069.jpg
/content/drive/MyDrive/Augmentation/valid/0/ISIC_0025350.jpg
/content/drive/MyDrive/Augmentation/valid/0/ISIC_0025825.jpg
/content/drive/MyDrive/Augmentation/valid/0/ISIC_0025948.jpg
/content/drive/MyDrive/Augmentation/valid/0/ISIC_0026525.jpg
/content/drive/MyDrive/Augmentation/valid/0/ISIC_0026992.jpg
/content/drive/MyDrive/Augmentation/valid/0/ISIC_0027172.jpg
/content/drive/MyDrive/Augmentation/valid/0/ISIC_0027753.jpg
/content/drive/MyDrive/Augmentation/valid/0/ISIC_0028314.jpg
/content/drive/MyDrive/Augmentation/valid/0/ISIC_0028499.jpg
/content/drive/MyDrive/Augmentation/valid/0/ISIC_0029059.jpg
/content/drive/MyDrive/Augmentation/valid/0/ISIC_0029541.jpg
/content/drive/MyDrive/Augmentation/valid/0/ISIC_0029713.jpg
/content/drive/MyDrive/A

In [11]:
dataset = torchvision.datasets.ImageFolder(root= G_path + '/01_Augmented_Imaged/test', transform=transforms.ToTensor())

for i in dataset.imgs:
    print(i[0])
    a = Image.open(i[0])
    a = np.asarray([np.asarray(a)], dtype=np.float64)
    input = torch.from_numpy(a).permute(0,3,1,2).to("cuda")
    
    out, latent_space_images = model.forward(input/255)
    Image.fromarray((latent_space_images[0].cpu().detach().numpy().transpose(1,2,0)*255).astype(np.uint8)).save(i[0].replace("Augmentation", "dataset_splitted_augmented"))

/content/drive/MyDrive/Augmentation/test/0/ISIC_0024539.jpg
/content/drive/MyDrive/Augmentation/test/0/ISIC_0024646.jpg
/content/drive/MyDrive/Augmentation/test/0/ISIC_0024843.jpg
/content/drive/MyDrive/Augmentation/test/0/ISIC_0025331.jpg
/content/drive/MyDrive/Augmentation/test/0/ISIC_0026132.jpg
/content/drive/MyDrive/Augmentation/test/0/ISIC_0026149.jpg
/content/drive/MyDrive/Augmentation/test/0/ISIC_0026212.jpg
/content/drive/MyDrive/Augmentation/test/0/ISIC_0026457.jpg
/content/drive/MyDrive/Augmentation/test/0/ISIC_0026729.jpg
/content/drive/MyDrive/Augmentation/test/0/ISIC_0027047.jpg
/content/drive/MyDrive/Augmentation/test/0/ISIC_0027650.jpg
/content/drive/MyDrive/Augmentation/test/0/ISIC_0028644.jpg
/content/drive/MyDrive/Augmentation/test/0/ISIC_0029067.jpg
/content/drive/MyDrive/Augmentation/test/0/ISIC_0029133.jpg
/content/drive/MyDrive/Augmentation/test/0/ISIC_0029417.jpg
/content/drive/MyDrive/Augmentation/test/0/ISIC_0029634.jpg
/content/drive/MyDrive/Augmentation/test